In [8]:
import nltk
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords
from keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
from keras.callbacks import EarlyStopping

data = pd.read_csv('train_data.csv')

# Pre-Processing the text 
def cleaning(df, stop_words):
    df['Text'] = df['Text'].astype(str)  # Convert to string
    df['Text'] = df['Text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Replacing the digits/numbers
    df['Text'] = df['Text'].str.replace(r'\d+', '', regex=True)
    # Removing stop words
    df['Text'] = df['Text'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))
    # Lemmatization
    df['Text'] = df['Text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
data = data.dropna(subset=['Text'])
data_cleaned = cleaning(data, stop_words)

tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_cleaned['Text'].values)
X = tokenizer.texts_to_sequences(data_cleaned['Text'].values)
X = pad_sequences(X)

# Encoding the target variable
Y = pd.get_dummies(data_cleaned['Sentiment']).values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Trainig
model = Sequential()
model.add(Embedding(500, 120, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(1024, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(512, activation='relu'))  # Changed LeakyReLU to relu
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


print(model.summary())

#Testing
score = model.evaluate(X_test, y_test)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 523s 2s/step - accuracy: 0.4905 - loss: 0.9991 - val_accuracy: 0.6441 - val_loss: 0.8265
Epoch 2/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 486s 2s/step - accuracy: 0.6578 - loss: 0.8019 - val_accuracy: 0.6600 - val_loss: 0.7926
Epoch 3/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 711s 3s/step - accuracy: 0.6747 - loss: 0.7621 - val_accuracy: 0.6598 - val_loss: 0.7953
Epoch 4/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 602s 2s/step - accuracy: 0.6851 - loss: 0.7432 - val_accuracy: 0.6620 - val_loss: 0.8054
Epoch 5/20
275/275 ━━━━━━━━━━━━━━━━━━━━ 619s 2s/step - accuracy: 0.6978 - loss: 0.7226 - val_accuracy: 0.6673 - val_loss: 0.7951


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 21, 120)        │        60,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 21, 120)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 1024)           │     4,689,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,828,779 (60.38 MB)

 Trainable params: 5,276,259 (20.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 10,552,520 (40.25 MB)

None
172/172 ━━━━━━━━━━━━━━━━━━━━ 56s 324ms/step - accuracy: 0.6670 - loss: 0.7791
Test Loss: 0.7770003080368042
Test Accuracy: 0.6679403185844421


In [9]:
model.save('model_big.h5')